In [2]:
from tkinter import *
import webbrowser
import os
import time
import numpy as np
from selenium import webdriver
import sys
import pandas as pd
from PIL import ImageTk,Image
import pafy
from datetime import datetime
import pyodbc
driver = webdriver.Chrome()
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=HARISH;'
                      'Database=breakapp;'
                      'Trusted_Connection=yes;')

root =Tk()
root.resizable(0,0)

st=webbrowser.get("C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s")


C = Canvas(root, bg="blue", height=250, width=300)
filename = PhotoImage(file = 'C:\\Users\\harish\\Downloads\\mana.png')
background_label = Label(root, image = filename)
background_label.place(relwidth=1, relheight=1)
C.grid()

pvvideo = []
pvlink=''
next_video = 1
playlist = []
history1 = []
start = 0
j=0

def add_url():
    global pvvideo
    global pvlink
    global url_entry
    pvlink=url_entry.get()
    lin = pafy.new(pvlink)
    pvvideo=lin.title
    cnxn=pyodbc.connect('Driver={SQL Server};'
                      'Server=HARISH;'
                      'Database=breakapp;'
                      'Trusted_Connection=yes;')
    cursor=cnxn.cursor()
    SQLCommand=("INSERT INTO playlist (pvname,plink) VALUES (?,?)")
    Values = [pvvideo,pvlink]
    cursor.execute(SQLCommand,Values)  
    cnxn.commit()
    url_entry.delete(0,END)
    
def start():
    sta=pd.read_sql_query("select * from playlist order by id desc",conn)
    b = np.array(sta.iloc[:,1:])
    driver.get(b[0][1])
    hhname=b[0][0]
    hhlink=b[0][1]
    cnxn=pyodbc.connect('Driver={SQL Server};'
                                'Server=HARISH;'
                                'Database=breakapp;'
                                'Trusted_Connection=yes;')
    cur=cnxn.cursor()
    SQLCommand=("INSERT INTO history (hname,hlink) VALUES (?,?)")
    Values = [hhname,hhlink]
    cur.execute(SQLCommand,Values)  
    cnxn.commit()

def close():
    driver.close()
    #driver.implicitly_wait(60)
    #start()
    #os.system('taskkill /im chrome.exe/f')
    root.destroy()
    
    
          
def play_next():
    pl=pd.read_sql_query("select * from playlist",conn)
    pl.head(3)
    b = np.array(pl.iloc[:,1:])
    global next_video
    global history1
    global image
    global j
    for i in range(len(b)):
        if j <= len(b):
            j+=1
            driver.get(b[j][1])
            hhname=b[j][0]
            hhlink=b[j][1]
            cnxn=pyodbc.connect('Driver={SQL Server};'
                                'Server=HARISH;'
                                'Database=breakapp;'
                                'Trusted_Connection=yes;')
            cur=cnxn.cursor()
            SQLCommand=("INSERT INTO history (hname,hlink) VALUES (?,?)")
            Values = [hhname,hhlink]
            cur.execute(SQLCommand,Values)  
            cnxn.commit()
            if j >= len(b)-1:
                j = -1
            break


def exit():
    cnxn=pyodbc.connect('Driver={SQL Server};'
                                'Server=HARISH;'
                                'Database=breakapp;'
                                'Trusted_Connection=yes;')
    curs=cnxn.cursor()
    SQLCommand1=("delete historyc")
    curs.execute(SQLCommand1) 
    SQLCommand2=("INSERT INTO historyc (id,hname,hlink) VALUES (1,'','')")
    curs.execute(SQLCommand2)
    cnxn.commit()
    driver.close()
    root.destroy()

    
def clear_history():
    cnxn=pyodbc.connect('Driver={SQL Server};'
                                'Server=HARISH;'
                                'Database=breakapp;'
                                'Trusted_Connection=yes;')
    curs=cnxn.cursor()
    SQLCommand=("delete history")
    curs.execute(SQLCommand) 
    cnxn.commit()
    
def show_playlist():
    global playlist
    root2 = Tk()
    root2.title('Current Playlist')
    spl=pd.read_sql_query("select * from playlist order by id desc",conn)
    spl.head(3)
    c = np.array(spl.iloc[:,1:])
    j=0
    k=0
    for i in range(len(c)):
        link2 = Label(root2,text=c[i][0], fg="blue")
        link2.pack()
        elink= lambda e:(lambda p: callback(e))
        link2.bind("<Button-1>",elink(c[i][1]))
        j+=1
        k+=1
    


class App():
    def __init__(self):
        self.root= Frame()
        self.label =Label(text="")
        self.label.place(x =440,y =22)
        self.update_clock()
        
    def update_clock(self):
        now = time.strftime("Time is : %H:%M:%S")
        self.label.configure(text=now)
        self.root.after(1000,self.update_clock)
        
def history():
    global history1
    root3 = Tk()
    root3.title('History')  
    sh=pd.read_sql_query("select id,convert(varchar,htime, 0)+'      '+hname as hname,hlink from history order by id desc",conn)
    sh.head(3)
    h = np.array(sh.iloc[:,1:])
    j=0
    k=0
    for i in range(len(h)):
        link3 = Label(root3,text=h[i][0], fg="blue")
        link3.pack()
        elink= lambda e:(lambda p: callback(e))
        link3.bind("<Button-1>",elink(h[i][1]))
        j+=1
        k+=1
    root3.mainloop()

def add_favorite():
        da=pd.read_sql_query("select * from historyc order by id desc",conn)
        aa=np.array(da.iloc[:,1:])
        vname=aa[0][0]
        link=aa[0][1]
        cnxn=pyodbc.connect('Driver={SQL Server};'
                                'Server=HARISH;'
                                'Database=breakapp;'
                                'Trusted_Connection=yes;')
        cur=cnxn.cursor()
        SQLCommand=("INSERT INTO videos (vname,link) VALUES (?,?)")
        Values = [vname,link]
        cur.execute(SQLCommand,Values)  
        cnxn.commit()
        
    
def shuffle():
    d = []
    sh=pd.read_sql_query("select * from playlist",conn)
    sh.head(3)
    f = np.array(sh.iloc[:,1:])
    for i in range(0,len(f)):
        d.append(i)
    x = np.array(d)
    np.random.shuffle(x)
    y = x.astype(int)
    for i in y:
        #st.open(f[i][1])
        driver.get(f[i][1])
        hhname=f[i][0]
        hhlink=f[i][1]
        cnxn=pyodbc.connect('Driver={SQL Server};'
                                'Server=HARISH;'
                                'Database=breakapp;'
                                'Trusted_Connection=yes;')
        cur=cnxn.cursor()
        SQLCommand=("INSERT INTO history (hname,hlink) VALUES (?,?)")
        Values = [hhname,hhlink]
        cur.execute(SQLCommand,Values)  
        cnxn.commit()
        break

def button_countdown(i, label):
    if i > 0:
        i -= 1
        label.set("Minutes Left : %d" %(i/60))
        root.after(1000, lambda: button_countdown(i, label))
    else:
        close()

def Play_display(i,j):
        vari=StringVar()
        da=pd.read_sql_query("select * from historyc order by id desc",conn)
        aa=np.array(da.iloc[:,1:])
        vari.set("NOW PLAYING : \n %s"%(aa[i][j]))
        label5=Label(root,textvariable=vari,justify=LEFT)
        root.after(1000,lambda: Play_display(i,j))
        label5.place(x =10,y =265)
    
def fav():
    filewin=Toplevel(root)
    df=pd.read_sql_query("select * from videos",conn)
    df.head(3)
    a = np.array(df.iloc[:,1:])
    j=0
    k=0
    for i in range(len(a)):
        link1 = Label(filewin,text=a[i][0], fg="blue")
        link1.pack()
        elink= lambda e:(lambda p: callback(e))
        link1.bind("<Button-1>",elink(a[i][1]))
        j+=1
        k+=1

def callback(url):
    driver.get(url)
    
menubar = Menu(root)
optmenu = Menu(menubar,tearoff=0)
menubar.add_cascade(label="File",menu=optmenu)
root.config(menu=menubar)
var=StringVar()
label=Label(root,textvariable=var,font='Arial')
label.place(x =100,y =20)
var.set("Hey! Welcome Back...")
app=App()
counter = 500
istart = PhotoImage(file ="C:\\Users\\harish\\Downloads\\sta.png")
ist= istart.subsample(3,3)
inext = PhotoImage(file ="C:\\Users\\harish\\Downloads\\nxt.png")
inxt= inext.subsample(4,4)
ishuffle = PhotoImage(file ="C:\\Users\\harish\\Downloads\\shu.png")
ishu= ishuffle.subsample(4,4)
iaddurl = PhotoImage(file ="C:\\Users\\harish\\Downloads\\url.png")
iurl= iaddurl.subsample(4,4)
iplay = PhotoImage(file ="C:\\Users\\harish\\Downloads\\ply.png")
iply= iplay.subsample(4,4)
iclear = PhotoImage(file ="C:\\Users\\harish\\Downloads\\clr.png")
iclr= iclear.subsample(4,4)
ihistory = PhotoImage(file ="C:\\Users\\harish\\Downloads\\his.png")
ihis= ihistory.subsample(4,4)
ifavorite = PhotoImage(file ="C:\\Users\\harish\\Downloads\\fav.png")
ifav= ifavorite.subsample(4,4)
iexit = PhotoImage(file ="C:\\Users\\harish\\Downloads\\ext.png")
iext= iexit.subsample(3,3)
iaftf= PhotoImage(file ="C:\\Users\\harish\\Downloads\\adf.png")
iadf=iaftf.subsample(3,3)

button_label =StringVar()
button_label.set(counter)
Button(root, textvariable=button_label).place(x=135,y=350)
button_countdown(counter, button_label)
Play_display(0,0)
root.geometry('680x460')
url_entry = Entry(root,width = 25,borderwidth =4)
button_add = Button(root,image=iurl,compound = LEFT,command = add_url)
start = Button(root,image=ist,compound = LEFT,command=start)
towork = Button(root, text="Back to Work", padx = 5, pady = 5,width = 17,height = 2, fg="blue",command=close)
exit = Button(root,image=iext,compound = LEFT, fg="red",command=exit)
favorites = Button(root,image=ifav,compound = LEFT, fg="purple", command =fav)
button_next = Button(root,image=inxt,compound = LEFT,fg="black", command = play_next)
button_history = Button(root,image=ihis,compound = LEFT, command = history)
button_shuffle = Button(root,image=ishu,compound = LEFT,fg="black", command = shuffle)
button_clear = Button(root,image=iclr,compound = LEFT, command = clear_history)
button_playlist = Button(root,image=iply,compound = LEFT, command = show_playlist)
button_adtofav= Button(root,image=iadf,compound=LEFT,command=add_favorite)

url_entry.place(x=40,y=130)
button_add.place(x=215,y=120)
button_playlist.place(x=400,y=120)
favorites.place(x=535,y=120)

start.place(x=120,y=199)
button_next.place(x=300,y=202)
button_shuffle.place(x=440,y=202)

button_adtofav.place(x=505,y=255)

button_history.place(x=300,y=320)
button_clear.place(x=440,y=320)

towork.place(x=400,y=385)
exit.place(x=120,y=385)




root.mainloop()